In [2]:
import pandas as pd

In [3]:
%cd ../../
%cd data
!ls

/Users/mubasir/Desktop/ONDC-hackathon
/Users/mubasir/Desktop/ONDC-hackathon/data
store_data                 stores_inventory-Sheet.csv


In [9]:
kolkata_store1_data = pd.read_csv("store_data/kol-str-1.csv")
kolkata_store1_data.head()

,date,product name,p _ category,store_id,city / District,eonomical crisis,health crisis,festivals in region,weather,population density,lot size
0,01/01/2020,Rice,Consumable,kol-str-1,Kolkatta,Low,Low,New year,Winter,24760,120.0
1,01/02/2020,Rice,Consumable,kol-str-1,Kolkatta,Low,Low,NaN,Summer,24760,140.0
2,01/03/2020,Rice,Consumable,kol-str-1,Kolkatta,Low,Low,NaN,Summer,24760,130.0
3,01/04/2020,Rice,Consumable,kol-str-1,Kolkatta,Low,Medium,Holi,Summer,24760,130.0
4,01/05/2020,Rice,Consumable,kol-str-1,Kolkatta,Low,Medium,Eid,Summer,24760,140.0


In [15]:
kolkata_store1_data.sort_values(by = ["date"], inplace=True)
kolkata_store1_data

,date,product name,p _ category,store_id,city / District,eonomical crisis,health crisis,festivals in region,weather,population density,lot size
0,01/01/2020,Rice,Consumable,kol-str-1,Kolkatta,Low,Low,New year,Winter,24760,120.00
60,01/01/2020,Sanitizer,Consumable,kol-str-1,Kolkatta,Low,Low,New year,Winter,24760,0.14
36,01/01/2020,Cold drink,Consumable,kol-str-1,Kolkatta,Low,Low,New year,Winter,24760,13.00
72,01/01/2020,Mask,Non Consumable,kol-str-1,Kolkatta,Low,Low,New year,Winter,24760,1.50
84,01/01/2020,Soap,Consumable,kol-str-1,Kolkatta,Low,Low,New year,Winter,24760,9.00
...,...,...,...,...,...,...,...,...,...,...,...
71,01/12/2020,Sanitizer,Consumable,kol-str-1,Kolkatta,Medium,Low,Christmas,Winter,24760,0.12
35,01/12/2020,Sugar,Consumable,kol-str-1,Kolkatta,Medium,Low,Christmas,Winter,24760,21.00
59,01/12/2020,Tea,Consumable,kol-str-1,Kolkatta,Medium,Low,Christmas,Winter,24760,18.00
11,01/12/2020,Rice,Consumable,kol-str-1,Kolkatta,Medium,Low,Christmas,Winter,24760,150.00


In [23]:
data = kolkata_store1_data.drop(columns=["store_id", "city / District "])
data

,date,product name,p _ category,eonomical crisis,health crisis,festivals in region,weather,population density,lot size
0,01/01/2020,Rice,Consumable,Low,Low,New year,Winter,24760,120.00
60,01/01/2020,Sanitizer,Consumable,Low,Low,New year,Winter,24760,0.14
36,01/01/2020,Cold drink,Consumable,Low,Low,New year,Winter,24760,13.00
72,01/01/2020,Mask,Non Consumable,Low,Low,New year,Winter,24760,1.50
84,01/01/2020,Soap,Consumable,Low,Low,New year,Winter,24760,9.00
...,...,...,...,...,...,...,...,...,...
71,01/12/2020,Sanitizer,Consumable,Medium,Low,Christmas,Winter,24760,0.12
35,01/12/2020,Sugar,Consumable,Medium,Low,Christmas,Winter,24760,21.00
59,01/12/2020,Tea,Consumable,Medium,Low,Christmas,Winter,24760,18.00
11,01/12/2020,Rice,Consumable,Medium,Low,Christmas,Winter,24760,150.00


In [36]:
train, test = data[:len(data)-18], data[len(data)-18:]

In [37]:
train

,date,product name,p _ category,eonomical crisis,health crisis,festivals in region,weather,population density,lot size
0,01/01/2020,Rice,Consumable,Low,Low,New year,Winter,24760,120.00
60,01/01/2020,Sanitizer,Consumable,Low,Low,New year,Winter,24760,0.14
36,01/01/2020,Cold drink,Consumable,Low,Low,New year,Winter,24760,13.00
72,01/01/2020,Mask,Non Consumable,Low,Low,New year,Winter,24760,1.50
84,01/01/2020,Soap,Consumable,Low,Low,New year,Winter,24760,9.00
...,...,...,...,...,...,...,...,...,...
56,01/09/2020,Tea,Consumable,High,Medium,NaN,Rainy,24760,13.00
32,01/09/2020,Sugar,Consumable,High,Medium,NaN,Rainy,24760,21.00
57,01/10/2020,Tea,Consumable,High,HIgh,"Durga puja , Kali Puja",Winter,24760,15.00
9,01/10/2020,Rice,Consumable,High,HIgh,"Durga puja , Kali Puja",Winter,24760,140.00


In [38]:
test

,date,product name,p _ category,eonomical crisis,health crisis,festivals in region,weather,population density,lot size
93,01/10/2020,Soap,Consumable,High,HIgh,"Durga puja , Kali Puja",Winter,24760,11.00
21,01/10/2020,Notebook,Non-Consumable,High,HIgh,"Durga puja , Kali Puja",Winter,24760,25.00
33,01/10/2020,Sugar,Consumable,High,HIgh,"Durga puja , Kali Puja",Winter,24760,26.00
105,01/10/2020,Umbrella,Non Consumable,High,HIgh,"Durga puja , Kali Puja",Winter,24760,0.10
45,01/10/2020,Cold drink,Consumable,High,HIgh,"Durga puja , Kali Puja",Winter,24760,5.00
81,01/10/2020,Mask,Non Consumable,High,HIgh,"Durga puja , Kali Puja",Winter,24760,9.90
94,01/11/2020,Soap,Consumable,High,Low,NaN,Winter,24760,9.00
10,01/11/2020,Rice,Consumable,High,Low,NaN,Winter,24760,120.00
46,01/11/2020,Cold drink,Consumable,High,Low,NaN,Winter,24760,6.00
82,01/11/2020,Mask,Non Consumable,High,Low,NaN,Winter,24760,1.90
